In [89]:
let ResolutionFolder = __SOURCE_DIRECTORY__

[<RequireQualifiedAccess>]
type Log =
| ChangeDir of string
| GoUp
| GoRoot
| File of int*string
| Dir of string

type AFile = {
    name: string
    size: int
}
type ADir = {
    name: string
    mutable parent: ADir option
    mutable dirs: ADir seq
    mutable files: AFile seq
} with
    member this.Size() =
        let files = (this.files |> Seq.sumBy( fun it -> it.size ))
        let dirs = (this.dirs |> Seq.sumBy (fun it -> it.Size()))
        files + dirs
    member this.Ls() =
        this.dirs |> Seq.map (fun it -> it.name) |> sprintf "%A"
    

let cd (dir:string) (scope:ADir) =
    scope.dirs
    |> Seq.tryFind(fun it -> it.name = dir)
    |> Option.orElse (Some scope)

let rec goRoot (scope: ADir option) =
    scope
    |> Option.bind (fun folder -> if folder.name = "/" then Some folder else folder.parent )
    |> Option.orElse ({ name = "/"; parent = None; dirs = Seq.empty; files = Seq.empty } |> Some)
   
let goUp (scope: ADir) =
    scope.parent |> Option.orElse (Some scope)

let addFile (size: int, name: string) (scope: ADir) =
    scope.files <- scope.files |> Seq.append [ { name = name; size = size } ]
    scope

let addDir (dir: string) (scope: ADir)  =
    scope.dirs <- 
        scope.dirs 
        |> Seq.append [ { name = dir; parent = Some scope; dirs = Seq.empty; files = Seq.empty } ]
    scope
let folder (state: ADir option) (line: Log) =
    match line with
    | Log.GoRoot -> goRoot state
    | Log.ChangeDir name -> state |> Option.bind (cd name)
    | Log.GoUp -> state |> Option.bind (goUp)
    | Log.File (size,name) -> state |> Option.map (addFile (size,name))
    | Log.Dir name -> state |> Option.map (addDir name)
    

let parseLog (line:string) =
    match line.Split(" ") with
    | [| "$"; "cd"; "/" |] -> Log.GoRoot |> Some
    | [| "$"; "cd"; ".." |] -> Log.GoUp |> Some
    | [| "$"; "cd"; dir |] -> Log.ChangeDir dir |> Some
    | [| "dir"; dir |] -> Log.Dir dir |> Some
    | [| size; file |] when (Int64.TryParse(size)) |> fst -> Log.File (size |> int ,file) |> Some
    | unknown -> 
        None

let rec find (fs:ADir) =
    seq{
        yield fs.Size()
        let children = 
            fs.dirs
            |> Seq.map find
            |> Seq.concat
        yield! children
    }

File.ReadLines(ResolutionFolder + "/input7.txt")
|> Seq.map parseLog
|> Seq.choose id
|> Seq.fold folder None
|> goRoot
|> Option.map (find >> Seq.filter (fun it -> it <= 100000) ) 

Value
"[ 23427191, 703677, 500913, 500913, 9319261, 2861769, 2832508, 976002, 580144, 150671, 258985, 62087, 1057988, 1039264, 717048, 243203, 141078, 139875, 139875, 2562 ... (more) ]"
